In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [3]:
# get EPL data for 2020-2021 season
df=pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")

In [4]:
# keep only the columns wwe want
df_clean=df[["Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HS","AS","HST","AST","HC","AC"]]

In [9]:
#Save the new df to as a .csv
df_clean.to_csv('C:/Users/ADMIN/Desktop/Soccer/Bet_System/Data/EPL2021_clean.csv')

In [12]:
# load in our clean df 
df_epl = pd.read_csv(r'C:\Users\ADMIN\Desktop\Soccer\Bet_System\Data\EPL2021_clean.csv', index_col=[0])

In [13]:
# Get team names from the two columns
home_teams = df_epl["HomeTeam"].unique()
away_teams = df_epl["AwayTeam"].unique()

In [14]:
# Combine the two numpy arrays into a list. Use "set" to get rid of duplicates
all_teams = list(set(np.append(home_teams, away_teams)))

In [15]:
# Test the idea by getting the first team in the list of teams
team = all_teams[0]

In [16]:
# create a DF with all the games for that first team
df_team = df_epl[(df_epl["HomeTeam"]==team) | (df_epl["AwayTeam"]==team)]

In [17]:
df_team.head(5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC
0,12/09/2020,Fulham,Arsenal,0,3,A,5,13,2,6,2,3
11,19/09/2020,Arsenal,West Ham,2,1,H,7,14,3,3,7,5
27,28/09/2020,Liverpool,Arsenal,3,1,H,21,4,8,3,7,3
34,04/10/2020,Arsenal,Sheffield United,2,1,H,6,6,5,2,2,4
40,17/10/2020,Man City,Arsenal,1,0,H,13,11,5,3,6,6


In [18]:
# set Date as index
df_team.set_index('Date', inplace=True)

In [20]:
# check
df_team.head(5)

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC
Date,,,,,,,,,,,
12/09/2020,Fulham,Arsenal,0,3,A,5,13,2,6,2,3
19/09/2020,Arsenal,West Ham,2,1,H,7,14,3,3,7,5
28/09/2020,Liverpool,Arsenal,3,1,H,21,4,8,3,7,3
04/10/2020,Arsenal,Sheffield United,2,1,H,6,6,5,2,2,4
17/10/2020,Man City,Arsenal,1,0,H,13,11,5,3,6,6


In [22]:
# Build Features (even though were just going to use two)

In [23]:
team_shots = df_team.apply( lambda x: x["HS"] if x["HomeTeam"] == team
    else x["AS"], axis=1).sum()

team_shots_ot = df_team.apply( lambda x: x["HST"] if x["HomeTeam"] == team
    else x["AST"], axis=1).sum()

team_goals = df_team.apply( lambda x: x["FTHG"] if x["HomeTeam"] == team
    else x["FTAG"], axis=1).sum()

team_shots_vs = df_team.apply( lambda x: x["AS"] if x["HomeTeam"] == team
    else x["HS"], axis=1).sum()

team_shots_ot_vs = df_team.apply( lambda x: x["AST"] if x["HomeTeam"] == team
    else x["HST"], axis=1).sum()

team_goals_vs = df_team.apply( lambda x: x["FTAG"] if x["HomeTeam"] == team
    else x["FTHG"], axis=1).sum()

team_points = df_team.apply(lambda x: 3 if ((x["HomeTeam"] == team and x["FTR"] == "H") or (x["AwayTeam"] == team and x["FTR"] == "A"))
    else (1 if x["FTR"] == "D" else 0), axis=1).sum()

In [37]:
# Define a function to do this for all teams:
# It takes 'df_epl' DataFrame and the 'team' list as arguments
# It returns the 'team_stats' dictionary with the team name and the sliced 
#'df_epl' DataFrame 'df_team'

In [24]:
def calc_feats(df, team):

    team_stats = dict()
    df_team = df_epl[(df_epl["HomeTeam"]==team) | (df_epl["AwayTeam"]==team)]
    team_stats["team"] = team
    return team_stats, df_team



In [25]:
# Call the function to test
calc_feats(df_epl, "Cardiff")

({'team': 'Cardiff'},
 Empty DataFrame
 Columns: [Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HS, AS, HST, AST, HC, AC]
 Index: [])

In [26]:
# Modify the function by adding the features we built and anly returning the 'team_stats' dictionary
def calc_feats(df, team):

    team_stats = dict()
    df_team = df_epl[(df_epl["HomeTeam"]==team) | (df_epl["AwayTeam"]==team)]
        
    team_shots = df_team.apply( lambda x: x["HS"] if x["HomeTeam"] == team
    else x["AS"], axis=1).sum()

    team_shots_ot = df_team.apply( lambda x: x["HST"] if x["HomeTeam"] == team
    else x["AST"], axis=1).sum()
    
    team_goals = df_team.apply( lambda x: x["FTHG"] if x["HomeTeam"] == team
    else x["FTAG"], axis=1).sum()

    team_shots_vs = df_team.apply( lambda x: x["AS"] if x["HomeTeam"] == team
    else x["HS"], axis=1).sum()

    team_shots_ot_vs = df_team.apply( lambda x: x["AST"] if x["HomeTeam"] == team
    else x["HST"], axis=1).sum()
    
    team_goals_vs = df_team.apply( lambda x: x["FTAG"] if x["HomeTeam"] == team
    else x["FTHG"], axis=1).sum()

    team_points = df_team.apply(lambda x: 3 if ((x["HomeTeam"] == team and x["FTR"] == "H") or (x["AwayTeam"] == team and x["FTR"] == "A"))
    else (1 if x["FTR"] == "D" else 0), axis=1).sum()
    
    team_stats["team"]=team
    team_stats["shots"] = team_shots
    team_stats["shots_ot"] = team_shots_ot
    #team_stats["goals"] = team_goals
    team_stats["shots_vs"] = team_shots_vs
    team_stats["shots_ot_vs"] = team_shots_ot_vs
    #team_stats["goals_vs"] = team_goals_vs
    team_stats["points"] = team_points
    
    return team_stats          

In [27]:
# test
calc_feats(df_epl, "Liverpool")

{'team': 'Liverpool',
 'shots': 608,
 'shots_ot': 214,
 'shots_vs': 333,
 'shots_ot_vs': 144,
 'points': 69}

In [28]:
# Put all teams stats into a list of dictionaries
all_teams_stats = []

In [29]:
for team in all_teams: 
    team_stats = calc_feats(df_epl, team)
    all_teams_stats.append(team_stats)

In [30]:
all_teams_stats

[{'team': 'Arsenal',
  'shots': 459,
  'shots_ot': 152,
  'shots_vs': 416,
  'shots_ot_vs': 132,
  'points': 61},
 {'team': 'Aston Villa',
  'shots': 511,
  'shots_ot': 187,
  'shots_vs': 538,
  'shots_ot_vs': 189,
  'points': 55},
 {'team': 'Man City',
  'shots': 604,
  'shots_ot': 219,
  'shots_vs': 285,
  'shots_ot_vs': 100,
  'points': 86},
 {'team': 'Leicester',
  'shots': 484,
  'shots_ot': 185,
  'shots_vs': 369,
  'shots_ot_vs': 139,
  'points': 66},
 {'team': 'Liverpool',
  'shots': 608,
  'shots_ot': 214,
  'shots_vs': 333,
  'shots_ot_vs': 144,
  'points': 69},
 {'team': 'Southampton',
  'shots': 423,
  'shots_ot': 166,
  'shots_vs': 428,
  'shots_ot_vs': 180,
  'points': 43},
 {'team': 'West Brom',
  'shots': 340,
  'shots_ot': 112,
  'shots_vs': 599,
  'shots_ot_vs': 246,
  'points': 26},
 {'team': 'Sheffield United',
  'shots': 325,
  'shots_ot': 99,
  'shots_vs': 542,
  'shots_ot_vs': 212,
  'points': 23},
 {'team': 'Burnley',
  'shots': 384,
  'shots_ot': 130,
  'shots_

In [31]:
# Turn the list of dictionaries into a DataFrame
df_all_stats = pd.DataFrame(all_teams_stats)

In [33]:
# Save the DF to a CSV file
df_all_stats.to_csv(r"C:\Users\ADMIN\Desktop\Soccer\Bet_System\Data\EPL2021_stats.csv")